#Project Imports

In [16]:
import boto3
import re
from sagemaker import get_execution_role
import sagemaker
import pandas as pd
import numpy as np
import json
import os
import shutil
import tarfile
import tensorflow as tf
import keras
from keras.models import model_from_json
from tensorflow import keras
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
import tensorflow.keras as layers
from sklearn.preprocessing import StandardScaler
from sagemaker.tensorflow.model import TensorFlowModel
from sklearn.metrics import r2_score
from keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.python.saved_model import builder
from tensorflow.python.saved_model.signature_def_utils  import predict_signature_def
from tensorflow.python.saved_model import tag_constants
from keras import backend as K
bucket = sagemaker.Session().default_bucket()

role = get_execution_role()

Getting The Data from the local environment

In [2]:
df=pd.read_csv("mushrooms.csv")

#The data File guaranted to have no null values.

In [3]:
df.isna().sum()

class                       0
cap-shape                   0
cap-surface                 0
cap-color                   0
bruises                     0
odor                        0
gill-attachment             0
gill-spacing                0
gill-size                   0
gill-color                  0
stalk-shape                 0
stalk-root                  0
stalk-surface-above-ring    0
stalk-surface-below-ring    0
stalk-color-above-ring      0
stalk-color-below-ring      0
veil-type                   0
veil-color                  0
ring-number                 0
ring-type                   0
spore-print-color           0
population                  0
habitat                     0
dtype: int64

#Looking at the mushroom Data.csv

In [4]:
df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


#number of unique vlaues at each cloumns

In [5]:
df.nunique()

class                        2
cap-shape                    6
cap-surface                  4
cap-color                   10
bruises                      2
odor                         9
gill-attachment              2
gill-spacing                 2
gill-size                    2
gill-color                  12
stalk-shape                  2
stalk-root                   5
stalk-surface-above-ring     4
stalk-surface-below-ring     4
stalk-color-above-ring       9
stalk-color-below-ring       9
veil-type                    1
veil-color                   4
ring-number                  3
ring-type                    5
spore-print-color            9
population                   6
habitat                      7
dtype: int64

We see some data have more than 2 # of unique data - so we need to use get dummies - using HOT encoding

In [6]:
df=pd.get_dummies(df, columns=['cap-shape','cap-color','cap-surface','odor','gill-color','stalk-root','stalk-surface-above-ring',
            'stalk-surface-below-ring','stalk-color-above-ring','stalk-color-below-ring','veil-color','ring-number','ring-type','spore-print-color',
            'population','habitat'])
df.head()

,class,bruises,gill-attachment,gill-spacing,gill-size,stalk-shape,veil-type,cap-shape_b,cap-shape_c,cap-shape_f,...,population_s,population_v,population_y,habitat_d,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w
0,p,t,f,c,n,e,p,0,0,0,...,1,0,0,0,0,0,0,0,1,0
1,e,t,f,c,b,e,p,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,e,t,f,c,b,e,p,1,0,0,...,0,0,0,0,0,0,1,0,0,0
3,p,t,f,c,n,e,p,0,0,0,...,1,0,0,0,0,0,0,0,1,0
4,e,f,f,w,b,t,p,0,0,0,...,0,0,0,0,1,0,0,0,0,0


#Here we encoding the Binary features and the class

In [7]:
le=LabelEncoder()
for item in df.columns:
  df[item]=le.fit_transform(df[item])

df.head()

,class,bruises,gill-attachment,gill-spacing,gill-size,stalk-shape,veil-type,cap-shape_b,cap-shape_c,cap-shape_f,...,population_s,population_v,population_y,habitat_d,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w
0,1,1,1,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,1,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
3,1,1,1,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
4,0,0,1,1,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


# we drop the target as y  from dataframe and the rest goes to x as features

In [8]:
x=df.drop(labels=['class'],axis=1)
y=df[['class']]

# Now we split the data to training and testing, 0.8 and 0.2 ratio respectively. 

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2,random_state=1)

#We are defining the model as Fucntion
where the model is Sequential consist 3 Dense layers.
The data shape (num of records , 112)
activation is 'relu'
and the last layer Sigmoid which make the output bettween 0 - 1 
since this Problem Binary classification.

In [10]:
def build_model():
    model = Sequential()
    model.add(Dense(32, activation = 'relu', input_dim = x_train.shape[1]))
    model.add(Dense(64, activation = 'relu'))
    model.add(Dense(1,  activation = 'sigmoid'))
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [11]:
model=build_model()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


#Now the fun begins with our Fiting method.
we are showing the logs "verbose=1".

In [12]:
model.fit(x_train,y_train,
          verbose=1,
          epochs=20,
          validation_split=.1
          )


Train on 5849 samples, validate on 650 samples
Epoch 1/20
5849/5849 [==============================] - 1s 248us/step - loss: 0.1937 - accuracy: 0.9244 - val_loss: 0.0185 - val_accuracy: 0.9954
Epoch 2/20
5849/5849 [==============================] - 1s 108us/step - loss: 0.0071 - accuracy: 0.9991 - val_loss: 0.0034 - val_accuracy: 1.0000
Epoch 3/20
5849/5849 [==============================] - 1s 103us/step - loss: 0.0019 - accuracy: 0.9998 - val_loss: 9.3978e-04 - val_accuracy: 1.0000
Epoch 4/20
5849/5849 [==============================] - 1s 96us/step - loss: 6.6096e-04 - accuracy: 1.0000 - val_loss: 4.6687e-04 - val_accuracy: 1.0000
Epoch 5/20
5849/5849 [==============================] - 1s 107us/step - loss: 3.5104e-04 - accuracy: 1.0000 - val_loss: 2.8882e-04 - val_accuracy: 1.0000
Epoch 6/20
5849/5849 [==============================] - 1s 101us/step - loss: 2.0318e-04 - accuracy: 1.0000 - val_loss: 2.2492e-04 - val_accuracy: 1.0000
Epoch 7/20
5849/5849 [===========================

We Got 100% for the accuracy!

#Evaluating the Model using r2_score from the sikit learn Library.
to see if the model overfiting or memorised the data

In [13]:
r2_score(y_test,model.predict(x_test))

0.999999913995359

Oh Nice! our Model is Perfect!
with 99% accuracy!

#We are ready to deploy the model to AWS cloud.
we make the directory Named keras_model as the usual protocal. 
and we save the weights to the same path.
then we write our model to Json.

In [14]:

!mkdir "keras_model"
save_path = "./keras_model/"

model.save_weights(os.path.join(save_path,"model-weights.h5"))

model_json = model.to_json()
with open(os.path.join(save_path,"model.json"), "w") as json_file:
    json_file.write(model_json)

#opening the Json file and loding the model to the loaded_model

In [17]:
json_file = open('keras_model/'+\
                 'model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json,custom_objects=\
        {"GlorotUniform": tf.keras.initializers.glorot_uniform})

In [18]:
#loading Weights
loaded_model.load_weights(\
    'keras_model/model-weights.h5')
print("Loaded model from disk")

Loaded model from disk


We make locally directory.
and the expected path for the model.

In [19]:
# Note: This directory structure will need to be followed - see notes 
# for the next section
model_version = '1'
export_dir = 'export/Servo/' + model_version

deleting anything leaving in that path

In [21]:

shutil.rmtree(export_dir)

# Build the Protocol Buffer SavedModel at 'export_dir'

In [22]:
build = builder.SavedModelBuilder(export_dir)

# Creating prediction signature to be used by TensorFlow Serving Predict API

In [23]:
signature = predict_signature_def(
    inputs={"inputs": loaded_model.input}, \
    outputs={"score": loaded_model.output})

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


In [24]:


with K.get_session() as sess:
    # Save the meta graph and variables
    build.add_meta_graph_and_variables(
        sess=sess, tags=[tag_constants.SERVING], \
        signature_def_map={"serving_default": signature})
    build.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: export/Servo/1/saved_model.pb


In [25]:

with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('export', recursive=True)

In [26]:
sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='model.tar.gz', \
                key_prefix='model')

In [27]:
#creating train.py
!touch train.py

In [28]:

sagemaker_model = TensorFlowModel(model_data = 's3://' + \
    sagemaker_session.default_bucket() + '/model/model.tar.gz',
    role = role,
    framework_version = '1.12',
    entry_point = 'train.py')

2.1.0 is the latest version of tensorflow that supports Python 2. Newer versions of tensorflow will only be available for Python 3.Please set the argument "py_version='py3'" to use the Python 3 tensorflow image.


Deploying the model to the Cloud.

In [ ]:
%%time
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type='ml.t2.medium')

---------------

In [ ]:
#EndPoint Name
predictor.endpoint

In [ ]:
endpoint_name = "sagemaker-tensorflow-2020-09-15-04-21-08-261"

In [ ]:
predictor=sagemaker.tensorflow.model.TensorFlowPredictor(\
        endpoint_name, sagemaker_session)

In [ ]:
#From inside the AWS 
client = boto3.client('runtime.sagemaker', region_name='us-east-1') 

In [ ]:
#Testing the deployed Predictor with dummiy data 

data = [[0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0]]

response = client.invoke_endpoint(EndpointName=endpoint_name, \
                                  Body=json.dumps(data))
response_body = response["Body"]
print(json.loads(response_body.read())['outputs']['score']['floatVal'])

And its ready to Go!